In [10]:
from openai import OpenAI
import base64
import os
import requests
import pandas as pd
import io
api_key = os.getenv('OPENAI_API_KEY')
client = OpenAI()

# An Example Page

A demonstration of parsing of one page. 

In [12]:
# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

# Path to your image
image_path = "../../data/primary_sources/Paris_1878_Italy_3.jpg"

In [13]:
# Getting the base64 string
base64_image = encode_image(image_path)

headers = {
  "Content-Type": "application/json",
  "Authorization": f"Bearer {api_key}"
}

In [21]:
payload = {
  "model": "gpt-4o",
  "messages": [
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "Parse this image to .csv with \t as a delimiter and the variables Name, Place, Description. Delimit the csv file by ```csv at the beginning and ``` at the end."
        },
        {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/jpeg;base64,{base64_image}"
          }
        }
      ]
    }
  ],
  "max_tokens": 2500
}

In [22]:
response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

print(response.json())

{'id': 'chatcmpl-9hd9jxc6p2F1fFbwLMNpkICglLIpS', 'object': 'chat.completion', 'created': 1720185647, 'model': 'gpt-4o-2024-05-13', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': "```csv\nName\tPlace\tDescription\nDe Sanctis.\t\tComposition musicale.\nFaa di Bruno (Chevalier F.)\tTurin\tPhasioscope, appareil pour expliquer la théorie des phases lunaires; appareil démonstratif du mouvement des nœuds et du périgée de la lune.\nFerrari (M°)\tBologne\tMesse des morts.\nFlorimo (Commandeur F.)\tNaples\tMéthode de chant.\nFurno (J.)\tNaples\tExercices pratiques d'harmonie adoptés au Conservatoire royal de musique de Naples; cantates de Niccolo Porpora, avec accompagnement de piano à l'usage des écoles de chant dudit Collège.\nGariboldi (Chevalier G. de Macerata)\tParis\tPetite école de la musique d'ensemble et d'accompagnement.\nGarzia\t\tChef de musique, Fantaisies musicales.\nGatti (D.)\t\tGrande méthode théorique pratique progressive pour trompette ou cornet en si bém

In [23]:
csv_content = response.json()['choices'][0]['message']['content']

# Remove markdown and extract CSV string
#csv_string = csv_content.split('```\n')[1].split('```')[0]
# Or:
csv_string = csv_content.split('```csv\n')[1].split('```')[0]

In [24]:
csv_string

"Name\tPlace\tDescription\nDe Sanctis.\t\tComposition musicale.\nFaa di Bruno (Chevalier F.)\tTurin\tPhasioscope, appareil pour expliquer la théorie des phases lunaires; appareil démonstratif du mouvement des nœuds et du périgée de la lune.\nFerrari (M°)\tBologne\tMesse des morts.\nFlorimo (Commandeur F.)\tNaples\tMéthode de chant.\nFurno (J.)\tNaples\tExercices pratiques d'harmonie adoptés au Conservatoire royal de musique de Naples; cantates de Niccolo Porpora, avec accompagnement de piano à l'usage des écoles de chant dudit Collège.\nGariboldi (Chevalier G. de Macerata)\tParis\tPetite école de la musique d'ensemble et d'accompagnement.\nGarzia\t\tChef de musique, Fantaisies musicales.\nGatti (D.)\t\tGrande méthode théorique pratique progressive pour trompette ou cornet en si bémol; l'école de perfectionnement, appendice.\nGuercia (Chevalier A.)\tNaples\tMéthode de chant adoptée par le Collège royal.\nKrakamp (Chevalier E.)\tNaples\tMéthode pour flûte cylindrique à la Boehm; méthode 

In [25]:
df = pd.read_csv(io.StringIO(csv_string), delimiter='\t')

In [26]:
df['Pdf'] = os.path.basename(image_path)

In [29]:
df.to_csv("../../data/gpt_ocr/" + os.path.basename(image_path) + ".csv", 
              sep="\t",
              index=False)  

In [30]:
df

,Name,Place,Description,Pdf
0,De Sanctis.,NaN,Composition musicale.,Paris_1878_Italy_3.jpg
1,Faa di Bruno (Chevalier F.),Turin,"Phasioscope, appareil pour expliquer la théori...",Paris_1878_Italy_3.jpg
2,Ferrari (M°),Bologne,Messe des morts.,Paris_1878_Italy_3.jpg
3,Florimo (Commandeur F.),Naples,Méthode de chant.,Paris_1878_Italy_3.jpg
4,Furno (J.),Naples,Exercices pratiques d'harmonie adoptés au Cons...,Paris_1878_Italy_3.jpg
5,Gariboldi (Chevalier G. de Macerata),Paris,Petite école de la musique d'ensemble et d'acc...,Paris_1878_Italy_3.jpg
6,Garzia,NaN,"Chef de musique, Fantaisies musicales.",Paris_1878_Italy_3.jpg
7,Gatti (D.),NaN,Grande méthode théorique pratique progressive ...,Paris_1878_Italy_3.jpg
8,Guercia (Chevalier A.),Naples,Méthode de chant adoptée par le Collège royal.,Paris_1878_Italy_3.jpg
9,Krakamp (Chevalier E.),Naples,Méthode pour flûte cylindrique à la Boehm; mét...,Paris_1878_Italy_3.jpg


# A function

A function to loop through all files and export jsons of raw outputs and ready-made csv's per page. 

In [31]:
def chatgpt_csv(file_path):
    # Encode the image
    # Getting the base64 string
    base64_image = encode_image(file_path)

    headers = {
      "Content-Type": "application/json",
      "Authorization": f"Bearer {api_key}"
    }
    # Construct query to ChatGPT
    payload = {
        "model": "gpt-4o",
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": "Parse this image to .csv with \t as a delimiter and the variables Name, Place, Description. Delimit the csv file by ```csv at the beginning and ``` at the end."
        },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{base64_image}"
                        }
                    }
                ]
    }
        ],
        "max_tokens": 2500
    }
    # Obtain response
    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
    # Obtain .csv content and parse
    csv_content = response.json()['choices'][0]['message']['content']
    # Remove markdown and extract CSV string
    try:
        csv_string = csv_content.split('```csv\n')[1].split('```')[0]
    except:
        csv_string = csv_content.split('```\n')[1].split('```')[0]
    # Parse the csv in a df
    df = pd.read_csv(io.StringIO(csv_string), delimiter='\t')
    # Add the filename
    df['Pdf'] = os.path.basename(file_path)
    # Export to directory
    df.to_csv("../../data/gpt_ocr/" + os.path.basename(file_path) + ".csv", 
              sep="\t",
              index=False)  
    
    
    

In [32]:
chatgpt_csv("../../data/primary_sources/Paris_1878_Italy_4.jpg")
# Debug from Italy 3, Italy 3 doens't parse correctly yet